In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *
from data_encoders import *
from parametrizations import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
np.random.seed(42)
x = np.array([[0.1, 1.2], [1.2, 0.2]])
y = np.array([[0.2], [1.2]])
theta = np.random.uniform(0, 2*np.pi, (2,1))

In [ ]:
def anzats(x, y, theta, backend, shots, verbose=False):
    features = qk.QuantumRegister(2, name="features")
    ancilla_features = qk.QuantumRegister(1, name="ancilla_feature")
    targets = qk.QuantumRegister(1, name="targets")
    ancilla_targets = qk.QuantumRegister(1, name="ancilla_target")
    ancilla_swap = qk.QuantumRegister(1, name="swap")
    classical = qk.ClassicalRegister(1)
    
    registers = [features, ancilla_features, targets, ancilla_targets, ancilla_swap, classical]
    circuit = qk.QuantumCircuit(*registers)

    circuit.h(ancilla)
    circuit.h(swap)

    circuit.cry(x[0,0], ancilla[0], features[0])
    circuit.cry(x[0,1], ancilla[0], features[1])

    circuit.x(ancilla[0])

    circuit.cry(x[1,0], ancilla[0], features[0])
    circuit.cry(x[1,1], ancilla[0], features[1])

    circuit.x(ancilla[0])

    circuit.cry(y[0,0], ancilla[1], targets[0])
    circuit.x(ancilla[1])
    circuit.cry(y[1,0], ancilla[1], targets[0])
    circuit.x(ancilla[1])

    circuit.barrier()

    circuit.cx(features[0], features[1])
    circuit.ry(theta[0,0], features[0])
    circuit.ry(theta[1,0], features[1])
    circuit.cx(features[0], features[1])

    circuit.barrier()

    circuit.cswap(swap, features[1], targets[0])
    circuit.cswap(swap, ancilla[0], ancilla[1])
    circuit.h(swap)
    circuit.measure(swap, classical)
    
    if verbose:
        print(circuit)

    job = qk.execute(circuit, backend, shots=shots)
    counts = job.result().get_counts(circuit)
    if "0" in counts:
        value = counts["0"] / shots
    else:
        value = 0
    
    return value

In [ ]:
backend = Aer.get_backend('qasm_simulator')
shots = 10000
for i in range(100):
    theta[0] += np.pi/2
    a = anzats(x, y, theta, backend, shots)
    theta[0] += -np.pi
    b = anzats(x, y, theta, backend, shots)
    theta[0] += np.pi/2
    dtheta1 = 1/(2*np.sqrt(2))*(a-b)

    theta[1] += np.pi/2
    c = anzats(x, y, theta, backend, shots)
    theta[1] += -np.pi
    d = anzats(x, y, theta, backend, shots)
    theta[1] += np.pi/2
    dtheta2 = 1/(2*np.sqrt(2))*(c-d)
    
    dtheta = np.array([[dtheta1], [dtheta2]])
    theta += 0.5*dtheta
    
    loss = anzats(x, y, theta, backend, shots)
    
    print(loss)  

In [15]:
x = np.array([[0.5, 0.5], [0.5, 2.5], [2.5, 0.5], [2.5, 2.5]])
y = np.array([[0.1], [0.2], [0.2], [0.2]])
theta = np.array([1., 2.])

In [16]:
features = qk.QuantumRegister(2, name="features")
ancilla_features = qk.QuantumRegister(2, name="ancilla_feature")
targets = qk.QuantumRegister(1, name="targets")
ancilla_targets = qk.QuantumRegister(2, name="ancilla_target")
ancilla_swap = qk.QuantumRegister(1, name="swap")
classical = qk.ClassicalRegister(1)

registers = [features, ancilla_features, targets, ancilla_targets, ancilla_swap, classical]

encoder = ParallelEncoder()
ansatz = Ansatz()
swaptest = SwapTest()

In [ ]:
circuit = qk.QuantumCircuit(*registers)

circuit = encoder(circuit, registers[:2], x)
circuit = encoder(circuit, registers[2:4], y)
circuit.barrier()

circuit = ansatz(circuit, features, theta)

circuit.barrier()
register_a = [features[-1]] + ancilla_features[:]
register_b = targets[:] + ancilla_targets[:]
circuit = swaptest(circuit, register_a, register_b, ancilla_swap)

In [ ]:
print(circuit)

In [31]:
backend = Aer.get_backend('qasm_simulator')
model = ParallelModel(backend=backend, shots=10000)

In [32]:
x = np.array([[0, 0], [1.5, 1.5]])
y = np.array([[0], [1.5]])

for i in range(100):
    grad = model.gradient(x, y)
    model.theta += -grad
    loss = model.loss(x,y)
    print(loss, grad, model.theta)

-0.6741 [-0.03556747  0.08513566] [1.03556747 1.91486434]
-0.6776 [-0.03500179  0.07424621] [1.07056926 1.84061813]
-0.6739 [-0.02177889  0.05946768] [1.09234815 1.78115045]
-0.6816 [-0.01180868  0.06066976] [1.10415683 1.72048069]
-0.686 [-0.01265721  0.05847773] [1.11681404 1.66200296]
-0.6819 [-0.01626346  0.0470226 ] [1.1330775  1.61498036]
-0.6947 [-0.00021213  0.0383959 ] [1.13328963 1.57658446]
-0.6901 [-0.00410122  0.05409367] [1.13739085 1.52249079]
-0.6921 [0.00311127 0.04101219] [1.13427958 1.4814786 ]
-0.6871 [0.0026163  0.03273904] [1.13166328 1.44873955]
-0.6911 [0.01859691 0.03344615] [1.11306637 1.4152934 ]
-0.6882 [0.01251579 0.01753625] [1.10055058 1.39775715]
-0.6935 [0.00890955 0.02460732] [1.09164104 1.37314984]
-0.6913 [0.01739483 0.03189052] [1.07424621 1.34125932]
-0.6904 [0.01994041 0.02899138] [1.0543058  1.31226794]
-0.7014 [0.02665793 0.02588011] [1.02764788 1.28638784]
-0.6898 [0.02503158 0.02771859] [1.0026163  1.25866925]
-0.6928 [0.02036468 0.02559727] [